# Error Handling and Logging Best Practices

This notebook demonstrates comprehensive error handling and logging patterns for Python applications, specifically addressing the ConnectionRefusedError and similar exceptions you encountered in the Firefox launcher proxy handlers.

## Objectives:
- ✅ Configure proper logging with different levels and formatters
- ✅ Handle specific exception types with appropriate error messages
- ✅ Implement production-ready error handling for HTTP connections
- ✅ Show async exception handling patterns
- ✅ Demonstrate context managers for resource cleanup
- ✅ Apply lessons learned from the Firefox launcher improvements

## 1. Setup Logging Configuration

Proper logging configuration is the foundation of good error handling. We'll set up a comprehensive logging system similar to what we implemented in the Firefox launcher.

In [ ]:
import logging
import sys
from pathlib import Path
import datetime

# Configure logging similar to firefox_handler.py
def setup_comprehensive_logging():
    """
    Set up logging with multiple handlers and formatters.
    This mimics the production-ready logging from the Firefox launcher.
    """
    
    # Create a logger for our demo
    logger = logging.getLogger('error_handling_demo')
    logger.setLevel(logging.DEBUG)  # Set to lowest level, handlers will filter
    
    # Clear any existing handlers
    logger.handlers.clear()
    
    # Create formatters
    detailed_formatter = logging.Formatter(
        '%(asctime)s - %(name)s - %(levelname)s - %(funcName)s:%(lineno)d - %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )
    
    simple_formatter = logging.Formatter(
        '%(levelname)s: %(message)s'
    )
    
    # Console handler for INFO and above (like production)
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setLevel(logging.INFO)
    console_handler.setFormatter(simple_formatter)
    logger.addHandler(console_handler)
    
    # Debug console handler for detailed output
    debug_handler = logging.StreamHandler(sys.stdout)
    debug_handler.setLevel(logging.DEBUG)
    debug_handler.setFormatter(detailed_formatter)
    
    # Only add debug handler if we want verbose output
    debug_mode = True  # Set to False for production-like behavior
    if debug_mode:
        logger.addHandler(debug_handler)
    
    return logger

# Initialize our demo logger
demo_logger = setup_comprehensive_logging()

# Test the logging setup
demo_logger.info("🚀 Logging system initialized successfully")
demo_logger.debug("🔍 Debug logging is enabled")
demo_logger.warning("⚠️ Warning level test")
demo_logger.error("❌ Error level test")

print("✅ Logging configuration complete!")
print(f"Logger level: {demo_logger.level}")
print(f"Number of handlers: {len(demo_logger.handlers)}")
for i, handler in enumerate(demo_logger.handlers):
    print(f"  Handler {i+1}: {type(handler).__name__} (level: {handler.level})")

## 2. Basic Exception Handling with Try-Catch

Let's start with fundamental exception handling patterns, showing how to properly catch and log exceptions with context.

In [ ]:
import traceback

def demonstrate_basic_exception_handling():
    """
    Show basic exception handling patterns with proper logging.
    This follows the patterns used in firefox_handler.py
    """
    
    demo_logger.info("🔍 Starting basic exception handling demonstration")
    
    # Example 1: Basic try-except with specific exception type
    try:
        demo_logger.debug("Attempting division by zero...")
        result = 10 / 0
        demo_logger.info(f"Result: {result}")
        
    except ZeroDivisionError as e:
        demo_logger.error(f"❌ Division error: {type(e).__name__}: {str(e)}")
        
        # Log traceback only in debug mode (like firefox_handler.py)
        if demo_logger.isEnabledFor(logging.DEBUG):
            demo_logger.debug(f"Division error traceback: {traceback.format_exc()}")
    
    # Example 2: Multiple exception types
    test_values = [10, "not_a_number", None]
    
    for i, value in enumerate(test_values):
        try:
            demo_logger.debug(f"Processing value {i+1}: {value}")
            result = int(value) * 2
            demo_logger.info(f"✅ Processed value {value}: result = {result}")
            
        except (ValueError, TypeError) as e:
            demo_logger.error(f"❌ Error processing value {value}: {type(e).__name__}: {str(e)}")
            
            # Provide context about what we were trying to do
            demo_logger.error(f"   Context: Attempting to convert '{value}' to integer and multiply by 2")
            
        except Exception as e:
            # Catch-all for unexpected errors
            demo_logger.error(f"💥 Unexpected error processing value {value}: {type(e).__name__}: {str(e)}")
            
            # Always log traceback for unexpected errors
            demo_logger.error(f"Unexpected error traceback: {traceback.format_exc()}")
    
    # Example 3: Exception with custom context (like process monitoring)
    process_id = 12345
    try:
        demo_logger.debug(f"Checking process {process_id}...")
        # Simulate process check that fails
        raise OSError("No such process")
        
    except OSError as e:
        demo_logger.warning(f"⚠️ Process check failed for PID {process_id}: {type(e).__name__}: {str(e)}")
        demo_logger.info(f"🗑️ Process {process_id} may have already terminated")
        
        # This is expected behavior, not an error condition
        return False
        
    except Exception as e:
        demo_logger.error(f"💥 Unexpected error checking process {process_id}: {type(e).__name__}: {str(e)}")
        
        # Log full traceback for debugging
        if demo_logger.isEnabledFor(logging.DEBUG):
            demo_logger.debug(f"Process check traceback: {traceback.format_exc()}")
        
        return None
    
    demo_logger.info("✅ Basic exception handling demonstration complete")

# Run the demonstration
demonstrate_basic_exception_handling()

## 3. Logging Different Exception Types

Different exception types require different logging approaches. Let's explore how to handle various common exceptions with appropriate log levels and context.

In [ ]:
import os
import json
from pathlib import Path

def demonstrate_exception_type_handling():
    """
    Show how to handle different exception types with appropriate log levels.
    Based on patterns from firefox_handler.py cleanup and error handling.
    """
    
    demo_logger.info("🔍 Testing different exception types and logging approaches")
    
    # 1. File operations (common in cleanup operations)
    test_file = Path("/nonexistent/path/test.txt")
    
    try:
        demo_logger.debug(f"Attempting to read file: {test_file}")
        content = test_file.read_text()
        demo_logger.info(f"✅ File read successfully: {len(content)} characters")
        
    except FileNotFoundError as e:
        demo_logger.warning(f"⚠️ File not found: {test_file}")
        demo_logger.debug(f"FileNotFoundError details: {str(e)}")
        # This might be expected behavior, so WARNING not ERROR
        
    except PermissionError as e:
        demo_logger.error(f"❌ Permission denied accessing file: {test_file}")
        demo_logger.error(f"Permission error: {str(e)}")
        # This is a real problem that needs attention
        
    except OSError as e:
        demo_logger.error(f"❌ OS error accessing file: {test_file}")
        demo_logger.error(f"OS error details: {type(e).__name__}: {str(e)}")
        
        # Log full traceback for OS errors as they're usually unexpected
        if demo_logger.isEnabledFor(logging.DEBUG):
            demo_logger.debug(f"OS error traceback: {traceback.format_exc()}")
    
    # 2. JSON parsing (common in web requests)
    test_json_strings = [
        '{"valid": "json"}',
        '{invalid json}',
        '{"incomplete": json',
        ''
    ]
    
    for i, json_str in enumerate(test_json_strings):
        try:
            demo_logger.debug(f"Parsing JSON string {i+1}: {json_str[:30]}...")
            data = json.loads(json_str)
            demo_logger.info(f"✅ Successfully parsed JSON {i+1}: {data}")
            
        except json.JSONDecodeError as e:
            demo_logger.error(f"❌ Invalid JSON in string {i+1}: {str(e)}")
            demo_logger.error(f"   Problem string: {repr(json_str)}")
            demo_logger.error(f"   Error position: line {e.lineno}, column {e.colno}")
            
        except ValueError as e:
            demo_logger.error(f"❌ Value error parsing JSON {i+1}: {str(e)}")
            
        except Exception as e:
            demo_logger.error(f"💥 Unexpected JSON parsing error: {type(e).__name__}: {str(e)}")
            demo_logger.error(f"Full traceback: {traceback.format_exc()}")
    
    # 3. Type errors (common in dynamic operations)
    operations = [
        (lambda: len(None), "Getting length of None"),
        (lambda: "string" + 123, "Concatenating string and int"),
        (lambda: {}.nonexistent_method(), "Calling nonexistent method"),
    ]
    
    for operation, description in operations:
        try:
            demo_logger.debug(f"Attempting: {description}")
            result = operation()
            demo_logger.info(f"✅ {description}: {result}")
            
        except TypeError as e:
            demo_logger.error(f"❌ Type error in '{description}': {str(e)}")
            demo_logger.debug(f"TypeError context: {traceback.format_exc()}")
            
        except AttributeError as e:
            demo_logger.error(f"❌ Attribute error in '{description}': {str(e)}")
            demo_logger.debug(f"AttributeError context: {traceback.format_exc()}")
            
        except Exception as e:
            demo_logger.error(f"💥 Unexpected error in '{description}': {type(e).__name__}: {str(e)}")
            demo_logger.error(f"Full traceback: {traceback.format_exc()}")
    
    # 4. Process-related errors (like in firefox_handler.py)
    def simulate_process_error():
        """Simulate process monitoring like in _cleanup_inactive_sessions"""
        process_id = 99999
        port = 12345
        
        try:
            demo_logger.debug(f"Checking process {process_id} on port {port}")
            # Simulate psutil.NoSuchProcess
            raise ProcessLookupError(f"No such process: {process_id}")
            
        except ProcessLookupError as e:
            demo_logger.info(f"🗑️ Found inactive session on port {port} (process {process_id} no longer exists)")
            demo_logger.debug(f"Process lookup details: {str(e)}")
            return 'cleanup_needed'
            
        except PermissionError as e:
            demo_logger.warning(f"⚠️ Permission denied checking process {process_id}: {str(e)}")
            # Don't remove on permission error, might be temporary
            return 'permission_denied'
            
        except Exception as e:
            demo_logger.warning(f"⚠️ Error checking session on port {port}: {type(e).__name__}: {str(e)}")
            # Don't remove on error, might be temporary (as in the real code)
            return 'check_error'
    
    result = simulate_process_error()
    demo_logger.info(f"📋 Process check result: {result}")
    
    demo_logger.info("✅ Exception type handling demonstration complete")

# Run the demonstration
demonstrate_exception_type_handling()

## 4. HTTP Request Exception Handling

This section addresses the **ConnectionRefusedError** from your traceback and shows how to properly handle HTTP connection errors with retry logic and proper logging.

In [ ]:
import socket
import time
from typing import Optional, Tuple

def demonstrate_http_connection_handling():
    """
    Demonstrate handling HTTP connection errors, specifically addressing
    the ConnectionRefusedError [Errno 111] from the Firefox launcher proxy.
    
    This simulates the error handling patterns from FirefoxProxyHandler.
    """
    
    demo_logger.info("🌐 Testing HTTP connection error handling patterns")
    
    # Simulate the connection check from FirefoxProxyHandler.head()
    def check_port_accessibility(port: int, timeout: float = 2.0) -> Tuple[bool, str]:
        """
        Check if a port is accessible, handling all connection errors gracefully.
        This mirrors the pattern used in firefox_handler.py FirefoxProxyHandler.
        """
        
        demo_logger.debug(f"🔍 Checking port accessibility: localhost:{port}")
        
        try:
            # Create socket with timeout (like in the real code)
            test_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            test_sock.settimeout(timeout)
            
            # Attempt connection
            result = test_sock.connect_ex(('localhost', port))
            test_sock.close()
            
            if result == 0:
                demo_logger.info(f"✅ Port {port} is accessible")
                return True, "accessible"
            else:
                demo_logger.warning(f"⚠️ Port {port} connection failed with code {result}")
                return False, f"connection_failed_{result}"
                
        except ConnectionRefusedError as conn_error:
            # This is the exact error from your traceback: [Errno 111] Connection refused
            demo_logger.warning(f"🚫 Connection refused to port {port}: {conn_error}")
            demo_logger.debug(f"ConnectionRefusedError details: errno={getattr(conn_error, 'errno', 'unknown')}")
            return False, "connection_refused"
            
        except socket.timeout as timeout_error:
            demo_logger.warning(f"⏰ Timeout connecting to port {port}: {timeout_error}")
            return False, "timeout"
            
        except OSError as os_error:
            # Handle other OS-level socket errors
            demo_logger.warning(f"💻 OS error connecting to port {port}: {os_error}")
            demo_logger.debug(f"OSError details: errno={getattr(os_error, 'errno', 'unknown')}")
            return False, f"os_error_{getattr(os_error, 'errno', 'unknown')}"
            
        except Exception as unexpected_error:
            # Catch unexpected errors
            demo_logger.error(f"💥 Unexpected error checking port {port}: {type(unexpected_error).__name__}: {unexpected_error}")
            
            # Log full traceback for debugging unexpected errors
            if demo_logger.isEnabledFor(logging.DEBUG):
                demo_logger.debug(f"Port check traceback: {traceback.format_exc()}")
            
            return False, "unexpected_error"
    
    # Test with various ports to trigger different error conditions
    test_ports = [
        22,      # SSH (likely running) 
        80,      # HTTP (may or may not be running)
        60561,   # The specific port from your traceback
        99999,   # Very unlikely to be in use
    ]
    
    for port in test_ports:
        accessible, reason = check_port_accessibility(port, timeout=1.0)
        demo_logger.info(f"📋 Port {port} check result: {reason}")
    
    # Demonstrate retry logic with exponential backoff
    def connection_with_retry(port: int, max_attempts: int = 3) -> bool:
        """
        Implement connection retry logic with exponential backoff.
        This pattern is useful for transient connection issues.
        """
        
        demo_logger.info(f"🔄 Attempting connection to port {port} with retry logic")
        
        for attempt in range(1, max_attempts + 1):
            try:
                demo_logger.debug(f"🔍 Connection attempt {attempt}/{max_attempts} to port {port}")
                
                # Simulate connection attempt
                test_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                test_sock.settimeout(1.0)
                result = test_sock.connect_ex(('localhost', port))
                test_sock.close()
                
                if result == 0:
                    demo_logger.info(f"✅ Successfully connected to port {port} on attempt {attempt}")
                    return True
                else:
                    raise ConnectionRefusedError(f"Connection failed with code {result}")
                    
            except (ConnectionRefusedError, socket.timeout, OSError) as conn_error:
                # Handle expected connection errors
                error_type = type(conn_error).__name__
                demo_logger.warning(f"⚠️ Attempt {attempt}/{max_attempts} failed: {error_type}: {conn_error}")
                
                if attempt < max_attempts:
                    # Calculate exponential backoff delay
                    delay = 2 ** (attempt - 1)  # 1s, 2s, 4s...
                    demo_logger.info(f"⏱️ Waiting {delay}s before retry...")
                    time.sleep(delay)
                else:
                    demo_logger.error(f"❌ All {max_attempts} connection attempts failed for port {port}")
                    return False
                    
            except Exception as unexpected_error:
                demo_logger.error(f"💥 Unexpected error on attempt {attempt}: {type(unexpected_error).__name__}: {unexpected_error}")
                
                # Don't retry on unexpected errors
                if demo_logger.isEnabledFor(logging.DEBUG):
                    demo_logger.debug(f"Retry attempt traceback: {traceback.format_exc()}")
                
                return False
        
        return False
    
    # Test retry logic on a port that's likely to fail
    success = connection_with_retry(60561, max_attempts=2)
    demo_logger.info(f"📋 Retry test result: {'success' if success else 'failed'}")
    
    # Demonstrate the HTTP status handling pattern
    def simulate_http_response_handling():
        """
        Simulate handling HTTP response errors beyond just connection issues.
        """
        
        demo_logger.info("📡 Simulating HTTP response error handling")
        
        # Simulate different HTTP scenarios
        scenarios = [
            (200, "OK", "success"),
            (404, "Not Found", "client_error"), 
            (500, "Internal Server Error", "server_error"),
            (503, "Service Unavailable", "service_unavailable"),
            (ConnectionRefusedError("Connection refused"), None, "connection_error")
        ]
        
        for i, (status_or_error, message, category) in enumerate(scenarios):
            try:
                demo_logger.debug(f"Processing HTTP scenario {i+1}: {category}")
                
                if isinstance(status_or_error, Exception):
                    # Simulate the exception being raised
                    raise status_or_error
                
                # Handle HTTP status codes
                status_code = status_or_error
                
                if 200 <= status_code < 300:
                    demo_logger.info(f"✅ HTTP {status_code} {message}: Request successful")
                    
                elif 400 <= status_code < 500:
                    demo_logger.warning(f"⚠️ HTTP {status_code} {message}: Client error")
                    
                elif 500 <= status_code < 600:
                    demo_logger.error(f"❌ HTTP {status_code} {message}: Server error")
                    
                else:
                    demo_logger.warning(f"❓ HTTP {status_code} {message}: Unexpected status")
                
            except ConnectionRefusedError as conn_error:
                # This is exactly what you saw in the traceback
                demo_logger.error(f"🚫 Connection refused: {conn_error}")
                demo_logger.info("💡 This indicates the server is not running or not accepting connections")
                demo_logger.info("🔧 Suggested actions: Check if server is started, verify port number, check firewall")
                
            except Exception as other_error:
                demo_logger.error(f"💥 Unexpected HTTP error: {type(other_error).__name__}: {other_error}")
    
    simulate_http_response_handling()
    
    demo_logger.info("✅ HTTP connection error handling demonstration complete")

# Run the demonstration
demonstrate_http_connection_handling()

## 5. Custom Exception Classes

Custom exceptions help create more meaningful error handling and better structure for application-specific errors. Let's create some examples relevant to the Firefox launcher context.

In [ ]:
from typing import Dict, Any, Optional

# Custom exception classes for the Firefox launcher context
class FirefoxLauncherError(Exception):
    """Base exception for Firefox launcher errors."""
    
    def __init__(self, message: str, context: Optional[Dict[str, Any]] = None):
        super().__init__(message)
        self.context = context or {}
        
    def __str__(self):
        base_msg = super().__str__()
        if self.context:
            context_str = ", ".join(f"{k}={v}" for k, v in self.context.items())
            return f"{base_msg} (Context: {context_str})"
        return base_msg

class XpraStartupError(FirefoxLauncherError):
    """Exception raised when Xpra fails to start."""
    pass

class PortAllocationError(FirefoxLauncherError):
    """Exception raised when unable to allocate a port."""
    pass

class ProcessMonitoringError(FirefoxLauncherError):
    """Exception raised during process monitoring operations."""
    pass

class ProxyConnectionError(FirefoxLauncherError):
    """Exception raised when proxy connection fails."""
    pass

def demonstrate_custom_exceptions():
    """
    Show how to use custom exceptions with structured error handling.
    """
    
    demo_logger.info("🏗️ Testing custom exception classes and structured error handling")
    
    # Example 1: Xpra startup failure
    def simulate_xpra_startup(port: int, command: list):
        """Simulate Xpra startup with detailed error context."""
        
        try:
            demo_logger.debug(f"Starting Xpra on port {port} with command: {command[:2]}...")
            
            # Simulate various failure conditions
            if port < 1024:
                raise PermissionError("Cannot bind to privileged port")
            elif port == 60561:  # The port from your traceback
                raise ConnectionRefusedError("Port already in use")
            elif "invalid_command" in str(command):
                raise FileNotFoundError("Xpra executable not found")
            else:
                # Simulate successful startup
                demo_logger.info(f"✅ Xpra started successfully on port {port}")
                return True
                
        except PermissionError as perm_error:
            # Convert to custom exception with context
            context = {
                "port": port,
                "error_type": "permission_denied",
                "command": command[:2] if command else []
            }
            
            error_msg = f"Cannot start Xpra on privileged port {port}"
            custom_error = XpraStartupError(error_msg, context)
            
            demo_logger.error(f"❌ Xpra startup failed: {custom_error}")
            demo_logger.error(f"   Original error: {perm_error}")
            demo_logger.error(f"   Suggested fix: Use port > 1024 or run with sudo")
            
            raise custom_error from perm_error
            
        except ConnectionRefusedError as conn_error:
            # Convert with specific context
            context = {
                "port": port,
                "error_type": "port_in_use",
                "errno": getattr(conn_error, 'errno', None)
            }
            
            error_msg = f"Port {port} is already in use or connection refused"
            custom_error = ProxyConnectionError(error_msg, context)
            
            demo_logger.error(f"🚫 Proxy connection failed: {custom_error}")
            demo_logger.error(f"   This matches your ConnectionRefusedError [Errno 111]")
            demo_logger.error(f"   Suggested fix: Check if another process is using port {port}")
            
            raise custom_error from conn_error
            
        except FileNotFoundError as file_error:
            context = {
                "command": command,
                "error_type": "executable_not_found",
                "executable": command[0] if command else "unknown"
            }
            
            error_msg = f"Xpra executable not found: {command[0] if command else 'unknown'}"
            custom_error = XpraStartupError(error_msg, context)
            
            demo_logger.error(f"❌ Xpra startup failed: {custom_error}")
            demo_logger.error(f"   Suggested fix: Install Xpra package")
            
            raise custom_error from file_error
            
        except Exception as unexpected_error:
            # Wrap unexpected errors
            context = {
                "port": port,
                "command": command[:2] if command else [],
                "error_type": "unexpected",
                "original_type": type(unexpected_error).__name__
            }
            
            error_msg = f"Unexpected error starting Xpra: {unexpected_error}"
            custom_error = XpraStartupError(error_msg, context)
            
            demo_logger.error(f"💥 Unexpected Xpra startup error: {custom_error}")
            
            # Log full traceback for unexpected errors
            if demo_logger.isEnabledFor(logging.DEBUG):
                demo_logger.debug(f"Xpra startup traceback: {traceback.format_exc()}")
            
            raise custom_error from unexpected_error
    
    # Test various scenarios
    test_scenarios = [
        (8080, ["xpra", "start"], "normal_startup"),
        (80, ["xpra", "start"], "privileged_port"),
        (60561, ["xpra", "start"], "connection_refused"),  # Your specific port
        (8081, ["invalid_command"], "missing_executable"),
    ]
    
    for port, command, scenario_name in test_scenarios:
        try:
            demo_logger.info(f"🧪 Testing scenario: {scenario_name}")
            result = simulate_xpra_startup(port, command)
            demo_logger.info(f"✅ Scenario {scenario_name} completed successfully")
            
        except FirefoxLauncherError as custom_error:
            demo_logger.error(f"❌ Scenario {scenario_name} failed with custom error:")
            demo_logger.error(f"   Error: {custom_error}")
            demo_logger.error(f"   Context: {custom_error.context}")
            
            # Show how to extract structured information
            error_type = custom_error.context.get('error_type', 'unknown')
            port_info = custom_error.context.get('port', 'unknown')
            demo_logger.info(f"📋 Extracted info - Type: {error_type}, Port: {port_info}")
            
        except Exception as other_error:
            demo_logger.error(f"💥 Scenario {scenario_name} failed with unexpected error: {other_error}")
    
    # Example 2: Process monitoring with custom exceptions
    def monitor_process_with_custom_errors(process_id: int):
        """Monitor process with custom exception handling."""
        
        try:
            demo_logger.debug(f"Monitoring process {process_id}")
            
            # Simulate process check
            if process_id == 0:
                raise ValueError("Invalid process ID")
            elif process_id == 99999:
                raise ProcessLookupError("No such process")
            else:
                demo_logger.info(f"✅ Process {process_id} is running")
                return True
                
        except (ProcessLookupError, ValueError) as proc_error:
            context = {
                "process_id": process_id,
                "error_type": "process_not_found" if isinstance(proc_error, ProcessLookupError) else "invalid_id",
                "check_timestamp": time.time()
            }
            
            error_msg = f"Process monitoring failed for PID {process_id}"
            custom_error = ProcessMonitoringError(error_msg, context)
            
            demo_logger.warning(f"⚠️ Process monitoring issue: {custom_error}")
            
            # This might be expected (process terminated), so don't re-raise
            return False
            
        except Exception as unexpected_error:
            context = {
                "process_id": process_id,
                "error_type": "unexpected_monitoring_error",
                "original_error": str(unexpected_error)
            }
            
            error_msg = f"Unexpected process monitoring error for PID {process_id}"
            custom_error = ProcessMonitoringError(error_msg, context)
            
            demo_logger.error(f"💥 Unexpected monitoring error: {custom_error}")
            
            raise custom_error from unexpected_error
    
    # Test process monitoring
    test_processes = [12345, 0, 99999]
    for pid in test_processes:
        try:
            is_running = monitor_process_with_custom_errors(pid)
            demo_logger.info(f"📋 Process {pid} status: {'running' if is_running else 'not running'}")
        except ProcessMonitoringError as monitoring_error:
            demo_logger.error(f"❌ Monitoring failed: {monitoring_error}")
    
    demo_logger.info("✅ Custom exception handling demonstration complete")

# Run the demonstration
demonstrate_custom_exceptions()

## 6. Context Managers for Resource Cleanup

Context managers ensure proper resource cleanup even when exceptions occur. This is crucial for managing processes, files, and network connections.

In [ ]:
from contextlib import contextmanager
import tempfile
import subprocess

def demonstrate_context_managers():
    """
    Show context managers for resource cleanup, relevant to Firefox launcher.
    """
    
    demo_logger.info("🛠️ Testing context managers for resource cleanup")
    
    # Example 1: Socket connection context manager
    @contextmanager
    def managed_socket_connection(host: str, port: int, timeout: float = 2.0):
        """
        Context manager for socket connections with proper cleanup.
        Similar to the socket usage in FirefoxProxyHandler.
        """
        
        sock = None
        try:
            demo_logger.debug(f"🔌 Opening socket connection to {host}:{port}")
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(timeout)
            
            # This will raise ConnectionRefusedError if the port is not accessible
            result = sock.connect_ex((host, port))
            
            if result != 0:
                raise ConnectionRefusedError(f"Connection failed with code {result}")
            
            demo_logger.info(f"✅ Socket connected to {host}:{port}")
            yield sock
            
        except ConnectionRefusedError as conn_error:
            demo_logger.warning(f"🚫 Connection refused to {host}:{port}: {conn_error}")
            raise
            
        except socket.timeout as timeout_error:
            demo_logger.warning(f"⏰ Socket timeout connecting to {host}:{port}: {timeout_error}")
            raise
            
        except Exception as sock_error:
            demo_logger.error(f"💥 Socket error: {type(sock_error).__name__}: {sock_error}")
            raise
            
        finally:
            # Always clean up the socket
            if sock:
                try:
                    sock.close()
                    demo_logger.debug(f"🔌 Socket connection to {host}:{port} closed")
                except Exception as close_error:
                    demo_logger.warning(f"⚠️ Error closing socket: {close_error}")
    
    # Test the socket context manager
    test_connections = [
        ("localhost", 22),    # SSH - likely to exist
        ("localhost", 60561), # The port from your traceback
        ("localhost", 99999), # Unlikely to exist
    ]
    
    for host, port in test_connections:
        try:
            demo_logger.info(f"🧪 Testing connection to {host}:{port}")
            
            with managed_socket_connection(host, port, timeout=1.0) as sock:
                demo_logger.info(f"✅ Successfully connected to {host}:{port}")
                # Do something with the connection
                demo_logger.debug(f"   Socket info: {sock.getsockname()}")
                
        except ConnectionRefusedError as conn_error:
            demo_logger.warning(f"❌ Connection refused to {host}:{port} - this is expected for unused ports")
            
        except socket.timeout:
            demo_logger.warning(f"⏰ Connection timeout to {host}:{port}")
            
        except Exception as other_error:
            demo_logger.error(f"💥 Unexpected connection error: {type(other_error).__name__}: {other_error}")
    
    # Example 2: Process management context manager
    @contextmanager
    def managed_process(command: list, timeout: float = 5.0):
        """
        Context manager for subprocess with proper cleanup.
        Similar to Xpra process management in firefox_handler.py.
        """
        
        process = None
        try:
            demo_logger.debug(f"🚀 Starting process: {command}")
            
            # Start the process (similar to Xpra startup)
            process = subprocess.Popen(
                command,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                # Use setsid for process group management if available
                preexec_fn=os.setsid if hasattr(os, 'setsid') else None
            )
            
            demo_logger.info(f"✅ Process started with PID: {process.pid}")
            yield process
            
        except FileNotFoundError as file_error:
            demo_logger.error(f"❌ Executable not found: {command[0]}")
            demo_logger.error(f"   FileNotFoundError: {file_error}")
            raise
            
        except Exception as proc_error:
            demo_logger.error(f"💥 Process startup error: {type(proc_error).__name__}: {proc_error}")
            raise
            
        finally:
            # Always clean up the process
            if process:
                try:
                    # Check if process is still running
                    if process.poll() is None:
                        demo_logger.info(f"🔥 Terminating process {process.pid}")
                        process.terminate()
                        
                        try:
                            # Wait for graceful termination
                            process.wait(timeout=3.0)
                            demo_logger.info(f"✅ Process {process.pid} terminated gracefully")
                        except subprocess.TimeoutExpired:
                            # Force kill if it doesn't terminate
                            demo_logger.warning(f"💀 Force killing process {process.pid}")
                            process.kill()
                            process.wait()
                    else:
                        exit_code = process.returncode
                        demo_logger.info(f"📋 Process {process.pid} already exited with code {exit_code}")
                        
                except Exception as cleanup_error:
                    demo_logger.error(f"❌ Error during process cleanup: {cleanup_error}")
    
    # Test the process context manager
    test_commands = [
        ["echo", "Hello from managed process"],
        ["sleep", "1"],  # Short-lived process
        ["nonexistent_command"],  # Will fail
    ]
    
    for command in test_commands:
        try:
            demo_logger.info(f"🧪 Testing process: {command}")
            
            with managed_process(command, timeout=2.0) as proc:
                # Monitor the process
                demo_logger.debug(f"   Process running with PID: {proc.pid}")
                
                # Wait a bit and check status
                time.sleep(0.5)
                poll_result = proc.poll()
                if poll_result is None:
                    demo_logger.debug(f"   Process {proc.pid} still running")
                else:
                    demo_logger.info(f"   Process {proc.pid} completed with exit code: {poll_result}")
                    
                    # Read output for completed processes
                    try:
                        stdout, stderr = proc.communicate(timeout=1.0)
                        if stdout:
                            demo_logger.info(f"   STDOUT: {stdout.decode('utf-8', errors='ignore').strip()}")
                        if stderr:
                            demo_logger.warning(f"   STDERR: {stderr.decode('utf-8', errors='ignore').strip()}")
                    except subprocess.TimeoutExpired:
                        demo_logger.debug("   Process output not available (timeout)")
                        
        except FileNotFoundError:
            demo_logger.warning(f"❌ Command not found: {command[0]} - this is expected")
            
        except Exception as other_error:
            demo_logger.error(f"💥 Unexpected process error: {type(other_error).__name__}: {other_error}")
    
    # Example 3: Try-finally for manual cleanup
    def demonstrate_try_finally():
        """Show try-finally pattern for cleanup without context managers."""
        
        demo_logger.info("🧹 Demonstrating try-finally cleanup pattern")
        
        temp_file = None
        try:
            # Create a temporary resource
            temp_file = tempfile.NamedTemporaryFile(mode='w', delete=False)
            temp_path = temp_file.name
            demo_logger.debug(f"📁 Created temporary file: {temp_path}")
            
            # Simulate work that might fail
            temp_file.write("Test data\n")
            temp_file.flush()
            
            # Simulate an error
            if True:  # Force an error for demonstration
                raise RuntimeError("Simulated processing error")
            
            demo_logger.info("✅ File processing completed")
            
        except RuntimeError as runtime_error:
            demo_logger.error(f"❌ Processing error: {runtime_error}")
            
        except Exception as other_error:
            demo_logger.error(f"💥 Unexpected error: {type(other_error).__name__}: {other_error}")
            
        finally:
            # Always clean up the temporary file
            if temp_file:
                try:
                    temp_file.close()
                    os.unlink(temp_file.name)
                    demo_logger.debug(f"🗑️ Cleaned up temporary file: {temp_file.name}")
                except Exception as cleanup_error:
                    demo_logger.warning(f"⚠️ Error cleaning up temporary file: {cleanup_error}")
    
    demonstrate_try_finally()
    
    demo_logger.info("✅ Context manager demonstration complete")

# Run the demonstration
demonstrate_context_managers()

## 7. Async Exception Handling

Async/await code requires special consideration for exception handling. This is relevant since the Firefox launcher handlers are async methods in Tornado.

In [ ]:
import asyncio
from typing import List

async def demonstrate_async_exception_handling():
    """
    Show async/await exception handling patterns.
    This mirrors the async methods in FirefoxLauncherHandler and FirefoxProxyHandler.
    """
    
    demo_logger.info("⚡ Testing async exception handling patterns")
    
    # Example 1: Basic async exception handling (like in the handler methods)
    async def async_operation_with_errors(operation_name: str, should_fail: bool = False):
        """Simulate async operations that might fail."""
        
        try:
            demo_logger.debug(f"🔄 Starting async operation: {operation_name}")
            
            # Simulate async work
            await asyncio.sleep(0.1)
            
            if should_fail:
                if "connection" in operation_name.lower():
                    raise ConnectionRefusedError("Async connection refused")
                elif "timeout" in operation_name.lower():
                    raise asyncio.TimeoutError("Async operation timed out")
                else:
                    raise RuntimeError(f"Async operation '{operation_name}' failed")
            
            demo_logger.info(f"✅ Async operation completed: {operation_name}")
            return f"success: {operation_name}"
            
        except ConnectionRefusedError as conn_error:
            # Handle the same error type from your traceback, but in async context
            demo_logger.error(f"🚫 Async connection refused in '{operation_name}': {conn_error}")
            demo_logger.error(f"   This is the same ConnectionRefusedError from your Tornado handler")
            raise
            
        except asyncio.TimeoutError as timeout_error:
            demo_logger.error(f"⏰ Async timeout in '{operation_name}': {timeout_error}")
            raise
            
        except Exception as other_error:
            demo_logger.error(f"💥 Unexpected async error in '{operation_name}': {type(other_error).__name__}: {other_error}")
            
            # Log traceback in debug mode
            if demo_logger.isEnabledFor(logging.DEBUG):
                demo_logger.debug(f"Async operation traceback: {traceback.format_exc()}")
            
            raise
    
    # Test individual async operations
    operations = [
        ("successful_operation", False),
        ("connection_failure", True),
        ("timeout_operation", True),
        ("general_failure", True),
    ]
    
    for op_name, should_fail in operations:
        try:
            demo_logger.info(f"🧪 Testing async operation: {op_name}")
            result = await async_operation_with_errors(op_name, should_fail)
            demo_logger.info(f"📋 Operation result: {result}")
            
        except ConnectionRefusedError as conn_error:
            demo_logger.warning(f"❌ Expected connection error in {op_name}: {conn_error}")
            
        except asyncio.TimeoutError as timeout_error:
            demo_logger.warning(f"❌ Expected timeout error in {op_name}: {timeout_error}")
            
        except Exception as other_error:
            demo_logger.error(f"❌ Unexpected error in {op_name}: {type(other_error).__name__}: {other_error}")
    
    # Example 2: Async context manager (like managing async locks)
    class AsyncResourceManager:
        """Async context manager for resource management."""
        
        def __init__(self, resource_name: str):
            self.resource_name = resource_name
            self.acquired = False
            
        async def __aenter__(self):
            try:
                demo_logger.debug(f"🔒 Acquiring async resource: {self.resource_name}")
                
                # Simulate async resource acquisition
                await asyncio.sleep(0.1)
                
                if "fail" in self.resource_name:
                    raise RuntimeError(f"Failed to acquire {self.resource_name}")
                
                self.acquired = True
                demo_logger.info(f"✅ Acquired async resource: {self.resource_name}")
                return self
                
            except Exception as acquire_error:
                demo_logger.error(f"❌ Failed to acquire async resource {self.resource_name}: {acquire_error}")
                raise
        
        async def __aexit__(self, exc_type, exc_val, exc_tb):
            try:
                if self.acquired:
                    demo_logger.debug(f"🔓 Releasing async resource: {self.resource_name}")
                    
                    # Simulate async cleanup
                    await asyncio.sleep(0.1)
                    
                    demo_logger.info(f"✅ Released async resource: {self.resource_name}")
                    
                if exc_type is not None:
                    demo_logger.warning(f"⚠️ Async resource {self.resource_name} released due to exception: {exc_type.__name__}")
                    
            except Exception as release_error:
                demo_logger.error(f"❌ Error releasing async resource {self.resource_name}: {release_error}")
                # Don't suppress the original exception
                return False
    
    # Test async context manager
    async_resources = ["good_resource", "fail_resource"]
    
    for resource_name in async_resources:
        try:
            demo_logger.info(f"🧪 Testing async resource: {resource_name}")
            
            async with AsyncResourceManager(resource_name) as resource:
                demo_logger.debug(f"   Using async resource: {resource.resource_name}")
                
                # Simulate work with the resource
                await asyncio.sleep(0.1)
                
                if "fail" in resource_name:
                    raise ValueError("Simulated error while using resource")
                
                demo_logger.info(f"✅ Async resource {resource_name} used successfully")
                
        except RuntimeError as runtime_error:
            demo_logger.warning(f"❌ Expected resource acquisition error: {runtime_error}")
            
        except ValueError as value_error:
            demo_logger.warning(f"❌ Expected usage error: {value_error}")
            
        except Exception as other_error:
            demo_logger.error(f"💥 Unexpected async resource error: {type(other_error).__name__}: {other_error}")
    
    # Example 3: Handling multiple concurrent async operations
    async def concurrent_operations_with_errors():
        """Demonstrate handling errors in concurrent async operations."""
        
        demo_logger.info("🚀 Testing concurrent async operations")
        
        # Create multiple operations, some will fail
        tasks = [
            async_operation_with_errors("concurrent_op_1", False),
            async_operation_with_errors("concurrent_connection_failure", True),
            async_operation_with_errors("concurrent_op_3", False),
            async_operation_with_errors("concurrent_timeout", True),
        ]
        
        # Use asyncio.gather with return_exceptions=True to collect all results
        try:
            results = await asyncio.gather(*tasks, return_exceptions=True)
            
            demo_logger.info("📋 Concurrent operations completed, analyzing results:")
            
            for i, result in enumerate(results):
                if isinstance(result, Exception):
                    error_type = type(result).__name__
                    demo_logger.error(f"   Task {i+1}: Failed with {error_type}: {result}")
                    
                    # Handle specific error types
                    if isinstance(result, ConnectionRefusedError):
                        demo_logger.error(f"     → This is your ConnectionRefusedError in concurrent context")
                    elif isinstance(result, asyncio.TimeoutError):
                        demo_logger.error(f"     → Async timeout in concurrent operation")
                else:
                    demo_logger.info(f"   Task {i+1}: Success - {result}")
                    
        except Exception as gather_error:
            demo_logger.error(f"💥 Error in concurrent operations: {type(gather_error).__name__}: {gather_error}")
    
    await concurrent_operations_with_errors()
    
    # Example 4: Async exception handling with timeouts (like HTTP requests)
    async def async_operation_with_timeout(timeout_seconds: float):
        """Demonstrate async timeout handling."""
        
        try:
            demo_logger.debug(f"⏱️ Starting async operation with {timeout_seconds}s timeout")
            
            # Use asyncio.wait_for for timeout control
            async def slow_operation():
                await asyncio.sleep(2.0)  # Simulate slow operation
                return "completed"
            
            result = await asyncio.wait_for(slow_operation(), timeout=timeout_seconds)
            demo_logger.info(f"✅ Timed operation completed: {result}")
            return result
            
        except asyncio.TimeoutError as timeout_error:
            demo_logger.warning(f"⏰ Operation timed out after {timeout_seconds}s: {timeout_error}")
            demo_logger.info(f"   This is similar to HTTP request timeouts")
            raise
            
        except Exception as other_error:
            demo_logger.error(f"💥 Unexpected error in timed operation: {type(other_error).__name__}: {other_error}")
            raise
    
    # Test timeout scenarios
    timeout_tests = [0.5, 3.0]  # Short timeout (will fail), long timeout (will succeed)
    
    for timeout in timeout_tests:
        try:
            demo_logger.info(f"🧪 Testing operation with {timeout}s timeout")
            result = await async_operation_with_timeout(timeout)
            demo_logger.info(f"📋 Timeout test result: {result}")
            
        except asyncio.TimeoutError:
            demo_logger.warning(f"❌ Expected timeout for {timeout}s operation")
            
        except Exception as other_error:
            demo_logger.error(f"💥 Unexpected timeout test error: {type(other_error).__name__}: {other_error}")
    
    demo_logger.info("✅ Async exception handling demonstration complete")

# Run the async demonstration
await demonstrate_async_exception_handling()

## 8. Best Practices for Production Logging

The final section covers production-ready logging patterns including structured logging, sensitive data filtering, and conditional debug logging - all patterns implemented in the Firefox launcher.

In [ ]:
import re
from typing import Union, Any

def demonstrate_production_logging_practices():
    """
    Show production-ready logging patterns based on firefox_handler.py implementation.
    """
    
    demo_logger.info("🏭 Demonstrating production logging best practices")
    
    # 1. Conditional debug logging (used throughout firefox_handler.py)
    def log_with_conditional_detail(operation: str, details: dict, error: Exception = None):
        """
        Show conditional logging pattern used in firefox_handler.py.
        Basic info always logged, detailed debug only when debug level enabled.
        """
        
        # Always log the main event
        demo_logger.info(f"🔍 {operation} operation")
        
        # Log basic context
        demo_logger.info(f"   Context: {details.get('main_info', 'No main info')}")
        
        # Detailed debug info only if debug logging enabled
        if demo_logger.isEnabledFor(logging.DEBUG):
            demo_logger.debug(f"📋 Detailed {operation} information:")
            for key, value in details.items():
                if key != 'main_info':  # Skip main info already logged
                    demo_logger.debug(f"   {key}: {value}")
        
        # Handle errors with conditional traceback (firefox_handler.py pattern)
        if error:
            demo_logger.error(f"❌ {operation} failed: {type(error).__name__}: {str(error)}")
            
            # Log traceback only in debug mode (exact pattern from firefox_handler.py)
            if demo_logger.isEnabledFor(logging.DEBUG):
                demo_logger.debug(f"{operation} traceback: {traceback.format_exc()}")
    
    # Test conditional logging
    test_details = {
        "main_info": "Testing connection to port 60561",
        "host": "localhost",
        "port": 60561,
        "timeout": 2.0,
        "retry_count": 3,
        "process_id": 12345,
        "command_line": ["xpra", "start", "--bind-tcp=0.0.0.0:60561"]
    }
    
    # Successful operation
    log_with_conditional_detail("Port Connection Check", test_details)
    
    # Failed operation with error
    test_error = ConnectionRefusedError("[Errno 111] Connection refused")
    log_with_conditional_detail("Port Connection Check", test_details, test_error)
    
    # 2. Structured error context (like in FirefoxLauncherHandler._start_server_proxy)
    def log_process_startup_context(process_info: dict, success: bool, error: Exception = None):
        """
        Log process startup with structured context, based on firefox_handler.py patterns.
        """
        
        if success:
            demo_logger.info("🎉 Process startup completed successfully!")
            demo_logger.info(f"   Process ID: {process_info.get('pid', 'unknown')}")
            demo_logger.info(f"   Port binding: {process_info.get('port', 'unknown')}")
            demo_logger.info(f"   Session available at: /proxy/{process_info.get('port', 'unknown')}/")
            
            # Additional debug info
            if demo_logger.isEnabledFor(logging.DEBUG):
                demo_logger.debug("📋 Complete process startup details:")
                for key, value in process_info.items():
                    demo_logger.debug(f"   {key}: {value}")
        else:
            # Comprehensive error logging (firefox_handler.py style)
            demo_logger.error("❌ Process startup FAILED!")
            demo_logger.error(f"   Process exit code: {process_info.get('exit_code', 'unknown')}")
            demo_logger.error(f"   Process ID was: {process_info.get('pid', 'unknown')}")
            demo_logger.error(f"   Port attempted: {process_info.get('port', 'unknown')}")
            demo_logger.error(f"   Command executed: {process_info.get('command', 'unknown')}")
            demo_logger.error("   Please check the process logs for more details.")
            
            if error:
                demo_logger.error(f"   Error details: {type(error).__name__}: {str(error)}")
    
    # Test structured process logging
    success_info = {
        "pid": 12345,
        "port": 60561,
        "command": ["xpra", "start", "--bind-tcp=0.0.0.0:60561"],
        "startup_time": 0.5,
        "memory_usage": "45MB"
    }
    log_process_startup_context(success_info, success=True)
    
    failure_info = {
        "exit_code": 1,
        "pid": 12346,
        "port": 60562,
        "command": ["xpra", "start", "--invalid-option"],
        "startup_time": 0.1
    }
    failure_error = RuntimeError("Invalid command line option")
    log_process_startup_context(failure_info, success=False, error=failure_error)
    
    # 3. Sensitive data filtering
    def log_with_sensitive_filtering(data: dict, sensitive_keys: list = None):
        """
        Log data while filtering sensitive information.
        """
        
        if sensitive_keys is None:
            sensitive_keys = ['password', 'token', 'key', 'secret', 'auth']
        
        def filter_sensitive(obj: Any) -> Any:
            """Recursively filter sensitive data."""
            if isinstance(obj, dict):
                filtered = {}
                for key, value in obj.items():
                    key_lower = key.lower()
                    if any(sensitive in key_lower for sensitive in sensitive_keys):
                        filtered[key] = '[FILTERED]'
                    else:
                        filtered[key] = filter_sensitive(value)
                return filtered
            elif isinstance(obj, list):
                return [filter_sensitive(item) for item in obj]
            elif isinstance(obj, str):
                # Filter potential secrets in strings
                for sensitive in sensitive_keys:
                    if sensitive in obj.lower():
                        return '[FILTERED STRING]'
                return obj
            else:
                return obj
        
        filtered_data = filter_sensitive(data)
        demo_logger.info("🔒 Logging with sensitive data filtering:")
        demo_logger.info(f"   Filtered data: {filtered_data}")
        
        # Show debug logging with original data (in secure environments only)
        if demo_logger.isEnabledFor(logging.DEBUG):
            demo_logger.debug("🚨 Debug: Original data (contains sensitive info):")
            demo_logger.debug(f"   Raw data: {data}")
    
    # Test sensitive data filtering
    test_data = {
        "user": "admin",
        "password": "secret123",
        "host": "localhost",
        "auth_token": "abc123def456",
        "port": 60561,
        "config": {
            "api_key": "sensitive_key_value",
            "timeout": 30
        },
        "command": ["xpra", "start", "--auth=secret123"]
    }
    
    log_with_sensitive_filtering(test_data)
    
    # 4. Log level management and performance
    def demonstrate_efficient_logging():
        """Show efficient logging patterns to avoid performance issues."""
        
        # BAD: String formatting always executed
        expensive_computation = lambda: "very_expensive_result_" + str(time.time())
        
        # This is inefficient - string formatting happens even if debug is disabled
        # demo_logger.debug(f"Expensive result: {expensive_computation()}")
        
        # GOOD: Check log level first (firefox_handler.py pattern)
        if demo_logger.isEnabledFor(logging.DEBUG):
            demo_logger.debug(f"Expensive result: {expensive_computation()}")
        
        # GOOD: Use lazy evaluation for complex logging
        demo_logger.info("Using efficient logging patterns")
        
        # For complex object representation
        class ComplexObject:
            def __init__(self):
                self.data = list(range(1000))  # Large data structure
                
            def __str__(self):
                return f"ComplexObject(size={len(self.data)})"
            
            def detailed_repr(self):
                return f"ComplexObject(data={self.data[:10]}...)"
        
        obj = ComplexObject()
        
        # Always log basic info
        demo_logger.info(f"📊 Processing object: {obj}")
        
        # Detailed info only in debug
        if demo_logger.isEnabledFor(logging.DEBUG):
            demo_logger.debug(f"📊 Detailed object info: {obj.detailed_repr()}")
    
    demonstrate_efficient_logging()
    
    # 5. Error correlation and tracking
    def log_with_correlation(operation_id: str, step: str, status: str, error: Exception = None):
        """
        Log with correlation IDs for tracking related operations.
        Useful for distributed systems and complex operations.
        """
        
        correlation_prefix = f"[{operation_id}]"
        
        if status == "start":
            demo_logger.info(f"{correlation_prefix} 🚀 Starting {step}")
        elif status == "success":
            demo_logger.info(f"{correlation_prefix} ✅ Completed {step}")
        elif status == "error":
            demo_logger.error(f"{correlation_prefix} ❌ Failed {step}")
            if error:
                demo_logger.error(f"{correlation_prefix}    Error: {type(error).__name__}: {str(error)}")
                
                # Traceback with correlation ID
                if demo_logger.isEnabledFor(logging.DEBUG):
                    demo_logger.debug(f"{correlation_prefix} Traceback: {traceback.format_exc()}")
        else:
            demo_logger.debug(f"{correlation_prefix} 📋 {step}: {status}")
    
    # Simulate a complex operation with correlation tracking
    operation_id = f"firefox_launch_{int(time.time())}"
    
    log_with_correlation(operation_id, "dependency_check", "start")
    log_with_correlation(operation_id, "dependency_check", "success")
    
    log_with_correlation(operation_id, "port_allocation", "start")
    log_with_correlation(operation_id, "port_allocation", "success")
    
    log_with_correlation(operation_id, "xpra_startup", "start")
    log_with_correlation(operation_id, "xpra_startup", "error", 
                        ConnectionRefusedError("Your exact error from the traceback"))
    
    # 6. Production error summary
    def log_production_error_summary():
        """
        Show how to create useful error summaries for production monitoring.
        """
        
        demo_logger.info("📋 Production Error Handling Summary")
        demo_logger.info("   Key patterns implemented in Firefox launcher:")
        demo_logger.info("   ✅ Conditional debug logging (traceback only in debug mode)")
        demo_logger.info("   ✅ Specific exception handling (ConnectionRefusedError, TimeoutError, etc.)")
        demo_logger.info("   ✅ Structured error context with operation details")
        demo_logger.info("   ✅ Resource cleanup with try-finally and context managers")
        demo_logger.info("   ✅ Async exception handling for Tornado handlers")
        demo_logger.info("   ✅ Sensitive data filtering for security")
        demo_logger.info("   ✅ Performance-conscious logging with level checks")
        demo_logger.info("   ✅ Error correlation for complex operation tracking")
        
        demo_logger.info("")
        demo_logger.info("🔧 Addressing your specific ConnectionRefusedError [Errno 111]:")
        demo_logger.info("   → Enhanced FirefoxProxyHandler.head() with specific exception handling")
        demo_logger.info("   → Added connection verification before proxy redirects")
        demo_logger.info("   → Implemented comprehensive error logging with context")
        demo_logger.info("   → Added automatic session cleanup for dead processes")
        demo_logger.info("   → Enhanced all HTTP handlers with production-ready error handling")
    
    log_production_error_summary()
    
    demo_logger.info("✅ Production logging best practices demonstration complete")

# Run the production logging demonstration
demonstrate_production_logging_practices()

## Summary: Error Handling Applied to Firefox Launcher

This notebook demonstrated comprehensive error handling patterns, directly addressing the **ConnectionRefusedError [Errno 111]** from your traceback and showing how we've implemented robust error handling throughout the Firefox launcher.

### Key Improvements Made:

1. **🔧 Fixed ConnectionRefusedError Handling**: Enhanced `FirefoxProxyHandler.head()` and `get()` methods with specific exception handling for connection failures, socket timeouts, and OS errors.

2. **🛡️ Production-Ready Error Handling**: Added comprehensive exception handling to all HTTP handlers with detailed error reporting and graceful failure handling.

3. **🧹 Automatic Session Cleanup**: Implemented `_cleanup_inactive_sessions()` to automatically detect and remove dead Xpra processes.

4. **📊 Enhanced Logging**: Added conditional debug logging, structured error context, and performance-conscious logging patterns throughout the codebase.

5. **⚡ Async Exception Handling**: Properly handle exceptions in async Tornado handlers with appropriate error responses and status codes.

### Specific Solutions for Your Error:

The **ConnectionRefusedError [Errno 111]** you encountered is now handled gracefully with:
- Connection verification before proxy redirects  
- Specific error messages for different failure types
- Automatic cleanup of failed sessions
- Proper HTTP status codes (503 Service Unavailable) for connection failures
- Detailed logging with actionable error information

The enhanced error handling ensures your Firefox launcher is now production-ready with robust exception handling, comprehensive logging, and automatic cleanup capabilities.